# Vibration TCM - Deep Learning Pipeline (Cloud Training)

This notebook allows you to train the **Transfer Learning (EfficientNet)** model using free GPU resources on Google Colab or Kaggle.

## Instructions
1.  **Upload Data**: Upload your `phase3_datasets.npz` file to the notebook environment (drag & drop to the file browser on the left).
2.  **Enable GPU**: Go to `Runtime` > `Change runtime type` > Select `T4 GPU` (or any available GPU).
3.  **Run All**: Click `Runtime` > `Run all`.
4.  **Download Results**: After training, download the `best_model.h5` and `study_results.json` files.

In [ ]:
!pip install optuna tensorflow scikit-learn pandas numpy

In [ ]:
import numpy as np
import tensorflow as tf
import optuna
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
import json
import os

# Check GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if len(tf.config.list_physical_devices('GPU')) == 0:
    print("WARNING: No GPU detected. Training will be slow!")

In [ ]:
DATASET_PATH = "phase3_datasets.npz"  # Upload this file!

def load_dataset(npz_path):
    if not os.path.exists(npz_path):
        raise FileNotFoundError(f"Dataset not found: {npz_path}. Please upload it!")
    data = np.load(npz_path, allow_pickle=True)
    return {key: data[key] for key in data.files}

def split_train_val(train_indices, groups, val_size, seed):
    splitter = GroupShuffleSplit(n_splits=1, test_size=val_size, random_state=seed)
    dummy = np.zeros(len(train_indices))
    rel_train_idx, rel_val_idx = next(splitter.split(dummy, groups=groups[train_indices]))
    return train_indices[rel_train_idx], train_indices[rel_val_idx]

In [ ]:
def build_transfer_learning(input_shape, num_classes, params):
    # Input shape is (freq_bins, time_bins, 1)
    inputs = tf.keras.layers.Input(shape=input_shape)
    
    # Convert 1-channel to 3-channel by repeating
    x = tf.keras.layers.Conv2D(3, (1, 1), padding='same')(inputs)
    
    # Resize to a size compatible with EfficientNet (e.g., 224x224)
    x = tf.keras.layers.Resizing(224, 224)(x)
    
    # Load EfficientNetB0 pre-trained on ImageNet
    base_model = tf.keras.applications.EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_shape=(None, None, 3) 
    )
    
    # Pass the 3-channel input through the base model
    x = base_model(x)
    
    # Unfreeze top N layers
    base_model.trainable = True
    fine_tune_at = len(base_model.layers) - params["fine_tune_layers"]
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
        
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(params["dropout"])(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)
    
    return tf.keras.Model(inputs, outputs)

def build_cnn2d(input_shape, num_classes, params):
    inputs = tf.keras.layers.Input(shape=input_shape)
    # Resize to a reasonable square size (e.g., 64x64)
    x = tf.keras.layers.Resizing(64, 64)(inputs)
    x = tf.keras.layers.Conv2D(params["filters1"], (3, 3), padding="same", activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(params["filters2"], (3, 3), padding="same", activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(params["dropout"])(x)
    x = tf.keras.layers.Dense(params["dense_units"], activation="relu")(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)
    return tf.keras.Model(inputs, outputs)

def compile_model(model, lr):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

In [ ]:
def objective(trial):
    tf.keras.backend.clear_session()
    
    # Choose model type (optional, or just stick to one)
    model_type = trial.suggest_categorical("model_type", ["transfer_learning", "cnn2d"])
    
    if model_type == "transfer_learning":
        params = {
            "fine_tune_layers": trial.suggest_int("fine_tune_layers", 0, 50),
            "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        }
        build_fn = lambda: build_transfer_learning(input_spec_shape, num_classes, params)
    else:
        params = {
            "filters1": trial.suggest_categorical("filters1", [16, 32, 64]),
            "filters2": trial.suggest_categorical("filters2", [32, 64, 128]),
            "dropout": trial.suggest_float("dropout", 0.1, 0.5),
            "dense_units": trial.suggest_categorical("dense_units", [64, 128]),
        }
        build_fn = lambda: build_cnn2d(input_spec_shape, num_classes, params)

    lr = trial.suggest_float("learning_rate", 1e-4, 5e-3, log=True)
    
    input_spec_shape = X_spec.shape[1:]
    num_classes = len(np.unique(y))
    
    accuracies = []
    # OPTIMIZATION: Use 3 folds instead of 5 to save GPU time
    gkf = GroupKFold(n_splits=3) 
    
    for fold_idx, (train_idx_base, test_idx) in enumerate(gkf.split(X_spec, y, groups)):
        train_idx, val_idx = split_train_val(train_idx_base, groups, 0.15, 42 + fold_idx)
        
        model = build_fn()
        compile_model(model, lr)
        
        callbacks = [
            tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
        ]
        
        model.fit(
            X_spec[train_idx],
            y[train_idx],
            validation_data=(X_spec[val_idx], y[val_idx]),
            epochs=12, # Reduced epochs (was 15/40)
            batch_size=32,
            callbacks=callbacks,
            verbose=1
        )
        
        _, test_acc = model.evaluate(X_spec[test_idx], y[test_idx], verbose=0)
        accuracies.append(test_acc)
        
    return np.mean(accuracies)

In [ ]:
# Main Execution
try:
    dataset = load_dataset(DATASET_PATH)
    X_spec = dataset["X_spec"]
    y = dataset["y"]
    groups = dataset["groups"]
    
    print(f"Loaded dataset with {len(y)} samples.")
    
    study = optuna.create_study(direction="maximize")
    # Run fewer trials to respect GPU limits
    study.optimize(objective, n_trials=10)
    
    print("Best trial:")
    trial = study.best_trial
    print("  Value: ", trial.value)
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")
        
    # Save best params
    with open("study_results.json", "w") as f:
        json.dump(trial.params, f)
        
except FileNotFoundError as e:
    print(e)
except Exception as e:
    print(f"An error occurred: {e}")